In [1]:
import numpy as np
import pandas as pd
import psycopg2

In [5]:
import json

with open('config.json') as f:
    conf = json.load(f)
    host = conf['host']
    database = conf['database']
    user = conf['user']
    passw = conf['passw']

In [6]:
conn_str = "host={} dbname={} user={} password={}".format(host, database, user, passw)
conn = psycopg2.connect(conn_str)

In [7]:
LEO_query = "SELECT vic.victim_id, vic.incident_id, vic.victim_type_id, ty.victim_type_name,\
            vic.assignment_type_id, ass.assignment_type_name, vic.activity_type_id,\
            act.activity_type_name, vic.age_range_low_num, vic.age_range_high_num, vic.age_id,\
            age.age_name, vic.sex_code, vic.agency_data_year FROM nibrs_victim as vic\
            JOIN nibrs_victim_type as ty ON vic.victim_type_id = ty.victim_type_id JOIN nibrs_age as age\
            ON vic.age_id = age.age_id JOIN nibrs_assignment_type as ass ON\
            ass.assignment_type_id = vic.assignment_type_id JOIN nibrs_activity_type as act\
            ON act.activity_type_id = vic.activity_type_id;"

LEO_df = pd.read_sql(LEO_query, con=conn)

In [8]:
LEO_df.head()

,victim_id,incident_id,victim_type_id,victim_type_name,assignment_type_id,assignment_type_name,activity_type_id,activity_type_name,age_range_low_num,age_range_high_num,age_id,age_name,sex_code,agency_data_year
0,79849725,73437778,5,Law Enforcement Officer,3,One-Officer (Assisted),1,Responding to Disturbance Call,NaN,NaN,5,Age in Years,M,None
1,79846671,73435092,5,Law Enforcement Officer,3,One-Officer (Assisted),1,Responding to Disturbance Call,NaN,NaN,5,Age in Years,M,None
2,79846537,73434209,5,Law Enforcement Officer,3,One-Officer (Assisted),1,Responding to Disturbance Call,NaN,NaN,5,Age in Years,M,None
3,79927873,73516536,5,Law Enforcement Officer,2,One-Officer (Alone),1,Responding to Disturbance Call,NaN,NaN,5,Age in Years,M,None
4,79909815,73498313,5,Law Enforcement Officer,2,One-Officer (Alone),1,Responding to Disturbance Call,NaN,NaN,4,Unknown,M,None


In [9]:
LEO_df.corr()

,victim_id,incident_id,victim_type_id,assignment_type_id,activity_type_id,age_range_low_num,age_range_high_num,age_id
victim_id,1.000000,0.999976,NaN,-0.022414,-0.110137,-0.127053,0.078349,0.053487
incident_id,0.999976,1.000000,NaN,-0.022188,-0.109461,-0.127010,0.078386,0.053456
victim_type_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
assignment_type_id,-0.022414,-0.022188,NaN,1.000000,0.099734,0.208548,-0.025098,-0.118252
activity_type_id,-0.110137,-0.109461,NaN,0.099734,1.000000,0.098511,0.005380,-0.003533
age_range_low_num,-0.127053,-0.127010,NaN,0.208548,0.098511,1.000000,-0.153467,NaN
age_range_high_num,0.078349,0.078386,NaN,-0.025098,0.005380,-0.153467,1.000000,NaN
age_id,0.053487,0.053456,NaN,-0.118252,-0.003533,NaN,NaN,1.000000


In [20]:
Vic_query = "SELECT vic.victim_id, vic.incident_id, vic.victim_type_id, ty.victim_type_name,\
            vic.age_range_low_num, vic.age_range_high_num, vic.sex_code, oty.offense_name,\
            oty.offense_category_name FROM nibrs_victim as vic JOIN nibrs_victim_type as ty\
            ON vic.victim_type_id = ty.victim_type_id JOIN nibrs_offense as off ON\
            off.incident_id = vic.incident_id JOIN nibrs_offense_type as oty ON\
            off.offense_type_id = oty.offense_type_id JOIN nibrs_incident as inc ON\
            inc.incident_id = vic.incident_id WHERE vic.victim_type_id = 4 AND inc.incident_date\
            >'2015-12-31';"

Vic_df = pd.read_sql(Vic_query, con=conn)

In [21]:
Vic_df.head()

,victim_id,incident_id,victim_type_id,victim_type_name,age_range_low_num,age_range_high_num,sex_code,offense_name,offense_category_name
0,95389075,87281308,4,Individual,48.0,0.0,M,Burglary/Breaking & Entering,Burglary/Breaking & Entering
1,95384706,87276968,4,Individual,25.0,0.0,M,Pocket-picking,Larceny/Theft Offenses
2,95384554,87277933,4,Individual,51.0,0.0,M,Counterfeiting/Forgery,Counterfeiting/Forgery
3,95390552,87282475,4,Individual,36.0,0.0,F,Simple Assault,Assault Offenses
4,95383000,87275260,4,Individual,37.0,0.0,F,Simple Assault,Assault Offenses


In [22]:
Vic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164010 entries, 0 to 164009
Data columns (total 9 columns):
victim_id                164010 non-null int64
incident_id              164010 non-null int64
victim_type_id           164010 non-null int64
victim_type_name         164010 non-null object
age_range_low_num        160948 non-null float64
age_range_high_num       160948 non-null float64
sex_code                 164010 non-null object
offense_name             164010 non-null object
offense_category_name    164010 non-null object
dtypes: float64(2), int64(3), object(4)
memory usage: 11.3+ MB


In [23]:
Vic_df.offense_name.value_counts()

Simple Assault                                 31872
Theft From Motor Vehicle                       22275
All Other Larceny                              22203
Destruction/Damage/Vandalism of Property       19532
Burglary/Breaking & Entering                   16237
Aggravated Assault                              8180
Motor Vehicle Theft                             6957
Impersonation                                   4916
Intimidation                                    4625
Credit Card/Automated Teller Machine Fraud      4605
Robbery                                         3293
Theft of Motor Vehicle Parts or Accessories     3057
Theft From Building                             2795
False Pretenses/Swindle/Confidence Game         2645
Counterfeiting/Forgery                          2057
Drug/Narcotic Violations                        1404
Rape                                            1341
Fondling                                         959
Kidnapping/Abduction                          

In [29]:
test_query = "SELECT oty.offense_name, COUNT(inc.incident_id), date_part('year',inc.incident_date)\
              FROM nibrs_incident as inc JOIN nibrs_offense as off ON inc.incident_id = off.incident_id\
              JOIN nibrs_offense_type as oty ON off.offense_type_id = oty.offense_type_id GROUP BY\
              oty.offense_name, inc.incident_date;"

test_df = pd.read_sql(test_query, con=conn)

In [30]:
test_df.head()

,offense_name,count,date_part
0,Aggravated Assault,44,2010.0
1,Aggravated Assault,18,2010.0
2,Aggravated Assault,15,2010.0
3,Aggravated Assault,9,2010.0
4,Aggravated Assault,12,2010.0


In [36]:
test_df.groupby(['offense_name'])['count'].sum()

offense_name
Aggravated Assault                              43660
All Other Larceny                              198155
Arson                                            2411
Assisting or Promoting Prostitution               290
Betting/Wagering                                  207
Bribery                                            51
Burglary/Breaking & Entering                   153163
Counterfeiting/Forgery                          26533
Credit Card/Automated Teller Machine Fraud      34117
Destruction/Damage/Vandalism of Property       173804
Drug Equipment Violations                       53013
Drug/Narcotic Violations                       121501
Embezzlement                                     5282
Extortion/Blackmail                               103
False Pretenses/Swindle/Confidence Game         25813
Fondling                                         5744
Gambling Equipment Violation                       56
Human Trafficking, Commercial Sex Acts             24
Human Trafficki